In [1]:
import pandas as pd
import numpy as np

from utils.preprocess import *

[nltk_data] Downloading package stopwords to /Users/new/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/new/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
file_loc = "data/"

In [3]:
train_data = pd.read_csv(file_loc+"train.csv")
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
clean_data = clean(train_data.copy())
clean_data.head()

-------------
Cleaning data
-------------
Data cleaned!


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why edits made username hardcore m...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww he matches background colour i'm seeming...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really trying edit war it's guy co...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make real suggestions improvement...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir hero any chance remember page that's


In [5]:
clean_data = decontract(clean_data)
clean_data.head()

------------------
Decontracting text
------------------
Texts decontracted!


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean,decontracted
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why edits made username hardcore m...,explanation why edits made username hardcore m...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww he matches background colour i'm seeming...,d'aww he matches background colour i am seemin...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really trying edit war it's guy co...,hey man i am really trying edit war it is guy ...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make real suggestions improvement...,more i cannot make real suggestions improvemen...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir hero any chance remember page that's,you sir hero any chance remember page that is


In [6]:
clean_data = lemmatize(clean_data)
clean_data.head()

----------------
Lemmatizing text
----------------
Text lemmatized!


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean,decontracted,lemmatized
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why edits made username hardcore m...,explanation why edits made username hardcore m...,explanation why edit make username hardcore m...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww he matches background colour i'm seeming...,d'aww he matches background colour i am seemin...,d'aww he match background colour i be seeming...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really trying edit war it's guy co...,hey man i am really trying edit war it is guy ...,hey man i be really try edit war it be guy co...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make real suggestions improvement...,more i cannot make real suggestions improvemen...,more i cannot make real suggestion improvemen...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir hero any chance remember page that's,you sir hero any chance remember page that is,you sir hero any chance remember page that be


In [7]:
X_train, X_val, y_train, y_val = split_data(clean_data)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

------------------
Splitting the data
------------------
Splitting done!


((127656,), (31915,), (127656, 6), (31915, 6))